#Support Vector Machine (SVM)

## Linear SVM
* Hard Margin Classification <br>
If we strictly impose that all instances must be off the street and on the right side, this is called hard margin classification. There are two main issues with hard margin clas‐ sification. First, it only works if the data is linearly separable. Second, it is sensitive to outliers.
* Soft Margin Classification <br>
To avoid issues faced by Hard Margin Classification, use a more flexible model. The objective is to find a good balance between keeping the street as large as possible and limiting the margin violations (i.e., instances that end up in the middle of the street or even on the wrong side). This is called soft margin classification.

When creating an SVM model using Scikit-Learn, we can specify a number of hyperparameters. C is one of those hyperparameters. Let's try it out with Iris dataset.

In [7]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline(
    [
     ("scaler", StandardScaler()),
     ("linear_svc", LinearSVC(C=1, loss="hinge")),
    ]
)


In [8]:
svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [9]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

Instead of using the LinearSVC class, we could use the SVC class with a linear kernel. When creating the SVC model, we would write SVC(kernel="linear", C=1). Or we could use the SGDClassifier class, with SGDClassifier(loss="hinge", alpha=1/ (m*C)). This applies regular Stochastic Gradient Descent to train a linear SVM classifier. It does not converge as fast as the LinearSVC class, but it can be useful to handle online classification tasks or huge datasets that do not fit in memory (out-of-core training).

```
Note - The LinearSVC class regularizes the bias term, so you should center the training set first by subtracting its mean. 
This is automatic if you scale the data using the StandardScaler. Also make sure you set the loss hyperparameter to "hinge", 
as it is not the default value. Finally, for better performance, you should set the dual hyperparameter to False, 
unless there are more features than training instances.
```

## Non-Linear SVM
Although linear SVM classifiers are efficient and work surprisingly well in many cases, many datasets are not even close to being linearly separable. One approach to handling nonlinear datasets is to add more features, such as polynomial features in some cases this can result in a linearly separable dataset.

To implement this idea using Scikit-Learn, create a Pipeline containing a Polyno mialFeatures transformer , followed by a StandardScaler and a LinearSVC. Let’s test this on the moons dataset: this is a toy dataset for binary classification in which the data points are shaped as two interleaving half circles.

In [10]:
from sklearn.datasets import make_moons
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline(
    [
     ("poly_features", PolynomialFeatures(degree=3)),
     ("scaler", StandardScaler()),
     ("svm_clf", LinearSVC(C=10, loss="hinge"))
    ]
)

In [11]:
polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

### Polynomial Kernel
Adding polynomial features is simple to implement and can work great with all sorts of Machine Learning algorithms (not just SVMs). That said, at a low polynomial degree, this method cannot deal with very complex datasets, and with a high polyno‐ mial degree it creates a huge number of features, making the model too slow.

When using SVM, we can use kernel trick. The kernel trick makes it possible to get the same result as if you had added many polynomial features, even with very high-degree polynomials, without actually having to add them. So there is no combinatorial explosion of the number of features because you don’t actually add any features. This trick is implemented by the SVC class. Let’s test it on the moons dataset:

In [12]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline(
    [
     ("scaler", StandardScaler()),
     ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
    ]
)

poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

This code trains an SVM classifier using a third-degree polynomial kernel. If your model is overfitting, you might want to reduce the polynomial degree. Conversely, if it is underfitting, you can try increasing it. The hyperparameter coef0 controls how much the model is influenced by highdegree polynomials versus low-degree polynomials.

### Similarity Features
Another technique to tackle nonlinear problems is to add features computed using a similarity function, which measures how much each instance resembles a particular landmark. Next, let’s define the similarity function to be the Gaussian Radial Basis Function (RBF).

Gaussian RBF - <br>
$
\phi_\gamma(x, l) = \exp(-\gamma\parallel x - l \parallel^2)
$

This is a bell-shaped function varying from 0 (very far away from the landmark) to 1 (at the landmark). You may wonder how to select the landmarks. The simplest approach is to create a landmark at the location of each and every instance in the dataset. Doing that creates many dimensions and thus increases the chances that the transformed training set will be linearly separable. The downside is that a training set with m instances and n features gets transformed into a training set with m instances and m features (assum‐ ing you drop the original features). If your training set is very large, you end up with an equally large number of features.

### Gaussian RBF Kernel
Just like the polynomial features method, the similarity features method can be useful with any Machine Learning algorithm, but it may be computationally expensive to compute all the additional features, especially on large training sets. Once again the kernel trick does its SVM magic, making it possible to obtain a similar result as if you had added many similarity features. Let’s try the SVC class with the Gaussian RBF kernel:

In [13]:
rbf_kernel_svm_clf = Pipeline(
    [
     ("scaler", StandardScaler()),
     ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ]
)
rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

## SVM Regression
The SVM algorithm is versatile: not only does it support linear and nonlinear classification, but it also supports linear and nonlinear regression. To use SVMs for regression instead of classification, the trick is to reverse the objective: instead of trying to fit the largest possible street between two classes while limiting margin violations, SVM Regression tries to fit as many instances as possible on the street while limiting margin violations (i.e., instances off the street). The width of the street is controlled by a hyperparameter, ϵ. 

Adding more training instances within the margin does not affect the model’s predictions; thus, the model is said to be ϵ-insensitive. You can use Scikit-Learn’s LinearSVR class to perform linear SVM Regression. 

To tackle nonlinear regression tasks, you can use a kernelized SVM model.

In [14]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

In [15]:
# The following code uses Scikit-Learn’s SVR class (which supports the kernel trick)
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

## Under the Hood

### Decision Function and Predictions
The linear SVM classifier model predicts the class of a new instance x by simply computing the decision function:

$
\hat{y} = \left\{ \begin{array}{rl}
0 &\mbox{ if $w^Tx + b < 0$} \\
1 &\mbox{ if $w^Tx + b \geq 0$}
       \end{array} \right.
$



### Training Objective
Consider the slope of the decision function: it is equal to the norm of the weight vector, $∥ w ∥$. If we divide this slope by 2, the points where the decision function is equal to $±1$ are going to be twice as far away from the decision boundary. In other words, dividing the slope by 2 will multiply the margin by 2.

So we want to minimize ∥ w ∥ to get a large margin. If we also want to avoid any margin violations (hard margin), then we need the decision function to be greater than 1 for all positive training instances and lower than –1 for negative training instances. If we define $t^{(i)} = –1$ for negative instances (if $y^{(i)} = 0$) and $t^{(i)} = 1$ for positive instances (if $y^{(i)} = 1$), then we can express this constraint as $t^{(i)}(w^⊺ x^{(i)} + b) ≥ 1$ for all
instances.


***Hard margin linear SVM classifier objective***<br>
$\underset{w, b}{\min} \frac{1}{2}w^Tw $<br>
subject to $ t^{(i)}(w^⊺ x^{(i)} + b) ≥ 1 $ for i = 1, 2, ..., m

To get the soft margin objective, we need to introduce a slack variable $\zeta^{(i)} \geq 0$ for each instance. It measures how much the ith instance is allowed to violate the margin. We now have two conflicting objectives: make the slack variables as small as possible to reduce the margin violations, and make $½ w^Tw$ as small as possible to increase the margin. This is where the C hyperparameter comes in: it allows us to define the tradeoff between these two objectives.

***Soft margin linear SVM classifier objective***<br>
$\underset{w, b, \zeta}{\min} \frac{1}{2}w^Tw + C\sum_{i=1}^m\zeta^{(i)}$<br>
subject to $ t^{(i)}(w^⊺ x^{(i)} + b) ≥ 1 - \zeta^{(i)} $ and $ \zeta^{(i)} \geq 0$ for i = 1, 2, ..., m



### Quadratic Programming
The hard margin and soft margin problems are both convex quadratic optimization problems with linear constraints. Such problems are known as Quadratic Programming (QP) problems. One way to train a hard margin linear SVM classifier is to use an off-the-shelf QP solver and pass it the preceding parameters. The resulting vector p will contain the bias term b = p0 and the feature weights wi = pi for i = 1, 2, ..., n. Similarly, you can use a QP solver to solve the soft margin problem.

### Dual Problem
The dual problem is faster to solve than the primal one when the number of training instances is smaller than the number of features. More importantly, the dual problem makes the kernel trick possible, while the primal does not.

### Kernelized SVMs
Refer book, too much maths.